# 計算機誤差

浮動小数点数は仮数部の桁数が限られているため，演算を行うたびに微小な誤差が発生する．

浮動小数点数演算から生じる誤差はしばしば**計算機誤差**と総称される．

## 丸め誤差
浮動小数点数どうしを四則演算して得られた値は浮動小数点数になるとは限らない．

In [ ]:
1.111111111^2

1.2345679009876545

正確な値は $1.111111111^2 = 1.234567900987654321$ であり，この場合は下4桁分が切り捨てられている．

このように演算結果の値に近い浮動小数点数への近似を **丸め** (rounding) といい，
その際に生じる誤差を **丸め誤差** (rounding error)  という．

丸め誤差は浮動小数点数の計算において，ほぼ必然的に発生する．
丸め方には，正/負の無限大の方向，原点方向，最近点への丸めなどがある．

## 情報落ち
浮動小数点数では，相対的に極めて小さな値（正確には絶対値が machine epsilonより小さな値）を加算しても，
丸めによって値が変化しないことがある．
これを情報落ちという．

倍精度のmachine epsilonは `eps(Float64)` or `eps(1.0)`などで取得できる．

In [ ]:
ϵ = eps(Float64)

2.220446049250313e-16

In [ ]:
1.0 + 1e-16

1.0

この例では情報落ちはやむを得ないが，
計算順序を変えることで情報落ちを回避できることもある．

In [ ]:
(1e-16 + 1.0) - 1.0

0.0

In [ ]:
1e-16 + (1.0 - 1.0)

1.0e-16

##  桁落ち
近い値の２つの浮動小数点数を引き算すると有効桁数が減少する．これを**桁落ち**と呼ぶ．

In [2]:
1.23456789 - 1.23456788

9.99999993922529e-9

有効数字9桁どうしの引き算をした結果，有効数字は1桁になってしまった．

これも計算の仕方を工夫することで回避できることがある．

## 桁落ちの回避：2次方程式の解の公式

 $b$ を正の実数とする．
2次方程式　$f(x) = x^2 - 2bx + 1 = 0$ の解は，解の公式より
$$
  \alpha := b - \sqrt{b^2 - 1}, \quad \beta := b + \sqrt{b^2 - 1} 
$$
と表される．
$b^2 \gg 1$の場合，
$\alpha$をこのまま計算すると， $\sqrt{b^2 - 1} \approx b$ であるから桁落ちが生じる．

In [ ]:
b = 1e8  # 1e8 = 10^8 
alpha = b - sqrt(b^2 - 1)

0.0

計算値は `0.0` であるが，これが解ではないことは代入してみれば明らかである．

次のように式変形してから計算すると，真値に近い値が得られる．
$$
b - \sqrt{b^2 - 1}  = \frac{1}{b + \sqrt{b^2 - 1}} 
$$


In [ ]:
b = 1e8
x = 1 / (b + sqrt(b^2 - 1))

5.0e-9

<div class="alert alert-warning">
Warning:  上の計算式は万能ではなく，b が負 かつ |b|が極めて大きい場合，桁落ちが発生する．
</div>

    